In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

import graphviz

print("tensorflow version %s" % tf.__version__)
print("numpy      version %s" % np.__version__)
print("matplotlib version %s" % mpl.__version__)

# Tensorflow Detials
Within tensorflow, a `tf.Graph` is a directed acyclic graph of operations (`tf.Operations`) - also known as a call graph. There is a default graph (`tf.get_default_graph()`) and any new operations are added to this graph. `tf.Operations` operate on a `tf.Tensor` and return a `tf.Tensor` ($f:X->Y$, where $X$ and $Y$ are tensors).

Evaluation of the graph requires a session (`tf.Session()`). One can construct a global session (or can use context managers so that the sessions are independent - this is useful if wanting to compute something on a separate device). In addition there is a special session `tf.InteractiveSession()` - designed for interactive python environments, and it won't require referencing the specific session.

In [ ]:
#Using a context manager
with tf.Session() as session:
    n_values = 32
    x = tf.linspace(-3., 3., 32)
    
    y = x.eval(session=session)
    z = session.run(x)
    print("x (and x[0]) type information:")
    print("------------------------------")
    print("type(x), x: ",type(x), x)
    print("type(x[0]), x[0]: ",type(x[0]), x[0])
    #type is purely a tensorflow object
    # ** It DOESN'T HAVE a Value **
    
    print()
    print("evaluated type information (explicit call to session.run or tf.eval):")
    print("---------------------------")
    print(type(y), y)
    #type is now a numpy object
    print(type(z), z)
    #type is now a numpy object
    
    print(type(session.run(x[0])),session.run(x[0]))
    #evaluates down to numpy.float32

In [ ]:
#Can't use an interactive session within a context;
try:
    with tf.InteractiveSession() as session:
        print("Inside context")
except AttributeError as e:
    print("InteractiveSession doesn't have an %s" % e)

In [ ]:
#Let's try running with the session (defined within the context manager)
try:
    session.run(x)
except RuntimeError as re:
    print("RuntimeError: %s" % re)
# (session is closed)

# tensorflow constants, variables (and lazy evaluation)
As mentioned earlier tensorflow stores computation as a call graph - or `dataflow graph` - and will not perform any calculations until the input data state is correct; - this is also known as **lazy evaluation**. There are a few advantages to this:
* tensorflow can be run on multiple computation devices (`/device:CPU:0` or `/cpu0`, `/device:GPU:i` or `/gpu:i` (`i`-th GPU device).
* lazy evaluation allows for arbitrary depth expression (recursive form);
* results that are never used, will never be evaluated

This contrasts the default `python` (and `numpy`) behaviour;

In [ ]:
# python behaviour:
print("python behaviour:")
print("-----------------")
x = 1
y = x + 1
print(y)

#numpy behaviour
print()
print("numpy behaviour:")
print("----------------")
x = np.linspace(-3., 3., 32)
y = x + 1
print(type(y),y)

print()
print("tensorflow behaviour:")
print("---------------------")
with tf.Session() as session:
    n_values = 32
    x = tf.linspace(-3., 3., 32, name='x')
    y = tf.Variable(x + 5, name='y')
    print(type(y), y)


# Explicit Initialization and Evaluation

In [ ]:
with tf.Session() as session:
    n_values = 32
    x = tf.linspace(-3., 3., 32, name='x')
    y = tf.Variable(x + 5, name='y')
    init_op = tf.initialize_all_variables()
    session.run(init_op)
    #initialize the 'variables'
    print(session.run(y))

In [ ]:
with tf.Session() as session:
    n_values = 32
    x = tf.linspace(-3., 3., 32, name='x')
    y = tf.Variable(x + 5, name='y')
    init_op = tf.initialize_all_variables()
    print("is y initialized: ", session.run(tf.is_variable_initialized(y)))
    #variable is not initialized to explicitly initialized
    # (init_op has to be run)

In [ ]:
with tf.Session() as session:
    n_values = 32
    x = tf.linspace(-3., 3., 32, name='x')
    y = tf.Variable(x + 5, name='y')
    print("is y initialized: ", session.run(tf.is_variable_initialized(y)))
    session.run(y.initializer)
    print("is y initialized: ", session.run(tf.is_variable_initialized(y)))
    #explicitly initialze 'y'
    print(session.run(y))

In [ ]:
with tf.Session() as session:
    n_values = 32
    x = tf.linspace(-3., 3., 32, name='x')
    y = tf.Variable(x + 5, name='y')
    z = tf.Variable(y - 3, name='z')
    print("is y initialized: ", session.run(tf.is_variable_initialized(y)))
    session.run(y.initializer)
    print("is y initialized: ", session.run(tf.is_variable_initialized(y)))
    #explicitly initialze 'y'
    print("is z initialized: ", session.run(tf.is_variable_initialized(z)))
    try:
        session.run(z)
    except tf.errors.FailedPreconditionError as FPE:
        print("z is uninitialized (as expected)\n %s" % FPE)
    #z is no initialized (only y is initialized)
    init_yz = tf.initialize_variables([y,z])
    #initialize variables in the list (and only those variables)
    session.run(init_yz)
    print("is z initialized: ", session.run(tf.is_variable_initialized(z)))
    

# Methods of Initializing Variables:
 * `tf.Variable.initializer`
 * `tf.initialize_variable([<list>])`
 * `tf.initialize_all_variables()`



In [ ]:
#Getting a list of all variables:
variables = tf.all_variables()
for var in variables:
    print(var.name)

#Notice the names:


# Variables:
Each time a variable is declared: `tf.Variable(..., name='y')` it is given a unique identifier (unless the variables are declared within a `variable_scope` with `reuse` set to true).

In [ ]:
#without shape information get_variable will fail;
try:
    tf.get_variable("y")
except ValueError as ve:
    print("Require shape information!\n %s" % ve)



# Variable Scope
Imagine you create a simple model for image filters (in this case only 2 convolutions). If you use `tf.Variable` you could define the following functions:
```
def image_filters(in_image):
    conv1_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv1_weights")
    conv1_biases = tf.Variable(tf.zeros([32]), name="conv1_biases")
    conv1 = tf.nn.conv2d(in_image, conv1_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + conv1_biases)

    conv2_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv2_weights")
    conv2_biases = tf.Variable(tf.zeros([32]), name="conv2_biases")
    conv2 = tf.nn.conv2d(relu1, conv2_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + conv2_biases)
```

This creates 4 variables per function call; `conv1_biases`, `conv2_biases`, `conv1_weights`, `conv2_weights`;

```
result1 = image_filter(image_1)
result2 = image_filter(image_2)
```
This would create two sets of variables;

In [ ]:
tf.reset_default_graph()
variables = tf.all_variables()
for var in variables:
    print(var.name)
#cleared the state;

def image_filters(in_image):
    conv1_weights = tf.Variable(tf.random_normal([3, 3, 1, 32]),
        name="conv1_weights")
    conv1_biases = tf.Variable(tf.zeros([32]), name="conv1_biases")
    conv1 = tf.nn.conv2d(in_image, conv1_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + conv1_biases)

    conv2_weights = tf.Variable(tf.random_normal([3, 3, 32, 32]),
        name="conv2_weights")
    conv2_biases = tf.Variable(tf.zeros([32]), name="conv2_biases")
    conv2 = tf.nn.conv2d(relu1, conv2_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + conv2_biases)

x = tf.placeholder(shape=[None, 784], dtype=tf.float32)
x_image = tf.reshape(x, [-1,28,28,1])

image_filters(x_image)
image_filters(x_image)

variables = tf.all_variables()
for var in variables:
    print(var.name)

# Common Solution (using a Dictionary)

```
variables_dict = {
    "conv1_weights": tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv1_weights")
    "conv1_biases": tf.Variable(tf.zeros([32]), name="conv1_biases")
    #... etc. ...
}

def my_image_filter(input_images, variables_dict):
    conv1 = tf.nn.conv2d(input_images, variables_dict["conv1_weights"],
        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + variables_dict["conv1_biases"])

    conv2 = tf.nn.conv2d(relu1, variables_dict["conv2_weights"],
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + variables_dict["conv2_biases"])

# The 2 calls to my_image_filter() now use the same variables
result1 = my_image_filter(image1, variables_dict)
result2 = my_image_filter(image2, variables_dict)
```
## PROBLEMS (Breaks encapsulation):
* The code that builds the graph must document the names, types, and shapes of variables to create (development issue).
* When the code changes, the callers may have to create more, or less, or different variables (development issue).

**One way to address the problem is to use classes to create a model, where the classes take care of managing the variables they need. For a lighter solution, not involving classes, TensorFlow provides a Variable Scope mechanism that allows to easily share named variables while constructing a graph.**

# Variable Scope:

Variable scope mechanism consists of 2 primary functions:
* `tf.get_variable(<name>, <shape>, <initializer>)`: CREATES or RETURNS a variable with a given name.
* `tf.variable_scope(<scope_name>)`: MANAGES namespaces for names passed to `tf.get_variable`

The function `tf.get_variable()` is used to get or create a variable instead of a direct call to `tf.Variable`. It uses an initializer instead of passing the value directly, as in `tf.Variable`. An initializer is a function that takes the shape and provides a tensor with that shape. Here are some initializers available in TensorFlow:

* `tf.constant_initializer(value)` initializes everything to the provided value,
* `tf.random_uniform_initializer(a, b)` initializes uniformly from [a, b],
* `tf.random_normal_initializer(mean, stddev)` initializes from the normal distribution with the given mean and standard deviation.

## Usage:

In [ ]:
tf.reset_default_graph()
variables = tf.all_variables()
for var in variables:
    print(var.name)

def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

def my_image_filter(in_input):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights",
        #                                      "conv1/biases".
        relu1 = conv_relu(in_input, [5, 5, 1, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights",
        #                                      "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

x = tf.placeholder(shape=[None, 784], dtype=tf.float32)
x_image = tf.reshape(x, [-1,28,28,1])

my_image_filter(x_image)

variables = tf.all_variables()
for var in variables:
    print(var.name)
# Now if my_image_filter is called again (we have a problem)
# Raises ValueError( ... name already exists in scope ...)

tf.reset_default_graph()

x = tf.placeholder(shape=[None, 784], dtype=tf.float32)
x_image = tf.reshape(x, [-1,28,28,1])

with tf.variable_scope("image_filters") as scope:
    result1 = my_image_filter(x_image)
    scope.reuse_variables()
    result2 = my_image_filter(x_image)
    
variables = tf.all_variables()
for var in variables:
    print(var.name) 